In [ ]:
import pod5
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
plt.rcdefaults()
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.size'] = 12
sys.path.insert(0, '/private/groups/brookslab/gabai/tools/NEMO/src/')

import importlib

from findWellPosGenes import *
importlib.reload(sys.modules['findWellPosGenes'])
importlib.reload(sys.modules['plot'])
from findWellPosGenes import *
from qc import *
from predict import *
from plot import *

## Supp table1: Data QC

In [1]:
# control
neg_nuclei = '/private/groups/brookslab/gabai/projects/Add-seq/data/ctrl/pod5/220308_ang_0.sorted.stats'
pos_nuclei_20 = '/private/groups/brookslab/gabai/projects/Add-seq/data/ctrl/pod5/220308_ang_20.sorted.stats'
pos_nuclei_100 = '/private/groups/brookslab/gabai/projects/Add-seq/data/ctrl/pod5/220308_ang_100.sorted.stats'
pos_nuclei_500 = '/private/groups/brookslab/gabai/projects/Add-seq/data/ctrl/pod5/220308_ang_500.sorted.stats'

# chrom nuclei
# rep1
nuclei_chrom_190429 = '/private/groups/brookslab/gabai/projects/Add-seq/data/chrom/mapping/20190429_chrom.sorted.stats'
# rep2
nuclei_chrom_210615 = '/private/groups/brookslab/gabai/projects/Add-seq/data/chrom/mapping/20210615_0802_shelansky_chrom.sorted.stats'
# rep3
chrom_nuclei_500 = '/private/groups/brookslab/gabai/projects/Add-seq/data/chrom/mapping/220331_Nuclei_Ang500_doradoBasecalled_movesOut.sorted.stats'

# chrom sphe
chrom_sphe_500 = '/private/groups/brookslab/gabai/projects/Add-seq/data/chrom/sphe/220524_500Ang_Spheroblast_dorado_movesOut_sorted.stats'

In [2]:
def read_samstats(stat, names = ["raw total sequences", "average quality", "average length", "reads mapped", "bases mapped (cigar)"]):
    infos = {}
    thisinfo = []
    
    with open(stat, 'r') as infile:
        for line in infile:
            line = line.strip().split('\t', 3)
            infos[line[1].split(':')[0]] = line[2]
    for i in names:
        thisinfo.append(infos[i])
    
    percent_mapped = int(infos["reads mapped"])/int(infos["raw total sequences"])
    thisinfo.append(str(round(percent_mapped*100, 3)))

    covf = stat.split('.stats')[0] + '.cov'
    covs = []
    with open(covf, 'r') as infile:
        header = infile.readlines(1)
        for line in infile:
            line = line.strip().split('\t')
            chr = line[0]
            if chr != 'chrM': covs.append(float(line[6]))
    
    avecovs = np.mean(covs)
    thisinfo.append(str(round(avecovs,3)))
    return thisinfo

In [ ]:
stats = [ neg_nuclei, pos_nuclei_20, pos_nuclei_100, pos_nuclei_500, 
         nuclei_chrom_190429, nuclei_chrom_210615, nuclei_chrom_190429, chrom_sphe_500]

labels = ['0uM negative control', '20uM positive control', '100uM positive control', '500uM positive control',
          'Nuclei rep1', 'Nuclei rep2', 'Nuclei rep3', 'Spheroplast']

In [ ]:
outf = open('../data/' + 'Supplemental_Table1.tsv', 'w')
outf.write('sample\tnumber of reads\taverage quality\taverage read length\t# of reads mapped\tnumber of bases mapped\tpercent of reads mapped (100%)\taverage coverage\n')
for i in range(len(stats)):
    sample = labels[i]
    info = read_samstats(stats[i])
    outf.write(f'{sample}\t' + '\t'.join(info) + '\n')
outf.close()

## Supp table2: well-positioned gene analysis

In [11]:
dt_outpath = '../data/'
fig_outpath = '../figures/'
window = 1200